In [9]:
## import the necessary libraries for time series modelling
!pip install pmdarima
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mp
import seaborn as sns
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_predict

import warnings
warnings.filterwarnings('ignore')

In [10]:
## read in train and test dataset
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [11]:
## setting the index as Date (train_df)

train_df.index = pd.to_datetime(train_df.Date , format = '%d/%m/%Y')
train_df.drop('Date',axis = 1, inplace = True)
train_df.head()

,T,RH,Gas,Value
Date,,,,
2016-03-15,12.020833,54.883334,CO,1053.200000
2016-03-16,9.833333,64.069791,CO,995.250000
2016-03-17,11.292708,51.107292,CO,1025.250000
2016-03-18,12.866319,51.530903,CO,1064.444444
2016-03-19,16.016667,48.843750,CO,1088.741667


In [12]:
## setting the index as Date (test_df)
test_df.index = pd.to_datetime(test_df.Date , format = '%d/%m/%Y')
test_df.drop(['Date','id'],axis = 1, inplace = True)
test_df.head()

,T,RH,Gas
Date,,,
2017-02-06,6.616667,51.734375,CO
2017-02-07,7.613194,43.930903,CO
2017-02-08,7.252083,50.966667,CO
2017-02-09,7.473611,50.166319,CO
2017-02-10,5.571875,46.604167,CO


In [14]:
## extract CO gas only
CO = train_df.loc[train_df['Gas'] == "CO"]
CO = CO.drop(columns=['Gas'])
CO.head()

,T,RH,Value
Date,,,
2016-03-15,12.020833,54.883334,1053.200000
2016-03-16,9.833333,64.069791,995.250000
2016-03-17,11.292708,51.107292,1025.250000
2016-03-18,12.866319,51.530903,1064.444444
2016-03-19,16.016667,48.843750,1088.741667


In [17]:
exogCO = CO[['T','RH']]

## fit the data into the SARIMAX model
arima_model = SARIMAX(CO['Value'], order=(1,0,1), trend='n',exog=exogCO).fit()
arima_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                  Value   No. Observations:                  328
Model:               SARIMAX(1, 0, 1)   Log Likelihood               -1909.097
Date:                Thu, 28 Jul 2022   AIC                           3828.195
Time:                        23:24:06   BIC                           3847.160
Sample:                    03-15-2016   HQIC                          3835.761
                         - 02-05-2017                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
T              0.8130      0.600      1.355      0.176      -0.363       1.989
RH             3.4195      0.499      6.850      0.000       2.441       4.398
ar.L1          0.9944      0.006    161.422      0.000       0.982       1.007
ma.L1         -0.0415      0.060     -0.688      0.491      -0.160       0.077
sigma2      6564.9000    462.564     14.192      0.000    5658.290    7471.510
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):                17.71
Prob(Q):                              0.94   Prob(JB):                         0.00
Heteroskedasticity (H):               1.92   Skew:                            -0.42
Prob(H) (two-sided):                  0.00   Kurtosis:                         3.77
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [16]:
## split into train and test data
train_data = CO[CO.index<='2016-08-20']
test_data = CO[CO.index>'2016-08-20']

trainExog = train_data[['T','RH']]
predExog = test_data[['T','RH']]

## define params
orders = [(1,0,1),(1,0,2),(1,0,3),
          (1,1,1),(1,1,2),(1,1,3),
          (2,0,1),(2,0,2),(2,0,3),
          (2,1,1),(2,1,2),(2,1,3)]

prediction_df = pd.DataFrame()
metrics_df = pd.DataFrame(columns=['model', 'RMSE_train', 'RMSE_test', 'AIC'])

for order in orders:
        print("trying "+str(order))
        try:
            arima_model = SARIMAX(train_data['Value'], order=order, trend='n', exog=trainExog).fit()
#             metrics_df = metrics_df.append({'model':order, 'AIC':arima_model.aic, 'BIC':arima_model.bic}, ignore_index=True)
            pred = arima_model.get_prediction(start='2016-03-15', end='2017-02-05', exog=predExog).predicted_mean
            prediction_df[f'{order}'] = pred

            rmse_train = round(mean_squared_error(train_data['Value'], pred[pred.index<='2016-08-20'], squared=False), 2)
            rmse_test = round(mean_squared_error(test_data['Value'], pred[pred.index>'2016-08-20'], squared=False), 2)
            metrics_df = metrics_df.append({'model': f'{order},{seasonal_order}', 'RMSE_train': rmse_train, 'RMSE_test': rmse_test, 'AIC': arima_model.aic, 'BIC':arima_model.bic}, ignore_index=True)
        
        except:
            print("LU decomposition error.")

trying (1, 0, 1)
LU decomposition error.
trying (1, 0, 2)
LU decomposition error.
trying (1, 0, 3)
LU decomposition error.
trying (1, 1, 1)
LU decomposition error.
trying (1, 1, 2)
LU decomposition error.
trying (1, 1, 3)
LU decomposition error.
trying (2, 0, 1)
LU decomposition error.
trying (2, 0, 2)
LU decomposition error.
trying (2, 0, 3)
LU decomposition error.
trying (2, 1, 1)
LU decomposition error.
trying (2, 1, 2)
LU decomposition error.
trying (2, 1, 3)
LU decomposition error.


In [8]:
metrics_df = metrics_df.sort_values('AIC')
metrics_df

,model,RMSE_train,RMSE_test,AIC,BIC
4,"(1, 1, 2)",NaN,NaN,3746.903290,3769.643051
9,"(2, 1, 1)",NaN,NaN,3747.343241,3770.083002
5,"(1, 1, 3)",NaN,NaN,3748.744521,3775.274243
10,"(2, 1, 2)",NaN,NaN,3748.767130,3775.296852
11,"(2, 1, 3)",NaN,NaN,3750.515593,3780.835274
3,"(1, 1, 1)",NaN,NaN,3757.255444,3776.205244
2,"(1, 0, 3)",NaN,NaN,3767.637844,3794.188939
1,"(1, 0, 2)",NaN,NaN,3787.194193,3809.952275
8,"(2, 0, 3)",NaN,NaN,3791.262749,3821.606858
0,"(1, 0, 1)",NaN,NaN,3828.194702,3847.159770
